In [1]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta, datetime
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import refinitiv.data as rd
import warnings
warnings.filterwarnings('ignore')
plt.style.use('dark_background')


rd.open_session()

<refinitiv.data.session.Definition object at 0x178405a90 {name='workspace'}>

In [3]:
def get_data(ric, s_date):

    spot_df = rd.get_history(
        ric, 'TRDPRC_1', interval='daily', start=s_date, count=10000)
    _e_date = datetime.fromtimestamp(datetime.timestamp(spot_df.index[-1]))
    e_date = _e_date + timedelta(days=1)
    e_date = e_date.strftime('%Y-%m-%d')
    _e_date = _e_date + timedelta(days=30)
    fce_date = _e_date.strftime('%Y-%m-%d')
    decomposition = seasonal_decompose(spot_df.TRDPRC_1, period=12)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    spot_df['spot_1diff'] = spot_df['TRDPRC_1'].diff()
    spot_df = spot_df[spot_df['spot_1diff'].notnull()]  # drop null rows
    lag_acf = acf(spot_df['spot_1diff'], nlags=50)
    lag_pacf = pacf(spot_df['spot_1diff'], nlags=50, method='ols')
    new_spot = spot_df['spot_1diff'].resample(
        'D').ffill()  # resample per day and fill the gaps
    new_spot = new_spot.bfill()
    new_spot = new_spot.astype('float')
    arma_model = ARIMA(new_spot, order=(1, 0, 1))
    results = arma_model.fit()
    residuals = pd.DataFrame(results.resid)
    predictions_ARIMA = pd.Series(results.fittedvalues, copy=True)
    predictions_ARIMA_cumsum = predictions_ARIMA.cumsum()
    predictions_ARIMA_final = pd.Series(
        spot_df['TRDPRC_1'].iloc[0], index=new_spot.index)
    predictions_ARIMA_final = predictions_ARIMA_final.add(
        predictions_ARIMA_cumsum, fill_value=0)
    new_spot = spot_df.TRDPRC_1.resample(
        'D', label='right').ffill().astype('float')
    es_model = ExponentialSmoothing(
        new_spot, trend='add', damped=False, seasonal='mul', seasonal_periods=30)
    es_results = es_model.fit()
    predicted_values = es_model.predict(
        params=es_results.params, start=e_date, end=fce_date)
    preds = pd.DataFrame(index=pd.date_range(
        start=e_date, end=fce_date), data=predicted_values, columns=['TRDPRC_1'])
    plt.figure(figsize=(16, 7))
    plt.plot(new_spot, label='Actual')
    plt.plot(preds, label='Forecast', color='pink')
    plt.legend(loc='best')
    plt.show()

    return preds

In [ ]:
rd.close_session()
